This notebook uses the Police Public Contact Survey (PPCS) dataset: `ppcs_cc.csv`.


### Libs and py-files


In [30]:
# importing libs and py files
%load_ext autoreload
%autoreload 2

import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import w8_logit as logit
import w8_probit as probit
import w8_estimation as est
from scipy.stats import norm
from scipy.stats import t

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load the data 

In [31]:
# Load the dataset
dat = pd.read_csv('ppcs_cc.csv')

# Inspect distribution of the target variable
print("\nDistribution of 'anyuseofforce_coded':")
print(dat['anyuseofforce_coded'].value_counts(normalize=True))

# Inspect value counts for categorical variables
categorical_vars = ["sblack", "shisp", "swhite", "sother", "smale", "omajblack", 
                    "omajhisp", "omajwhite", "omajother", "osplit", "inctype_lin", "sbehavior"]


Distribution of 'anyuseofforce_coded':
anyuseofforce_coded
0    0.994999
1    0.005001
Name: proportion, dtype: float64


### Table with summary statistics

In [32]:
# Define groups for demographic categories
group_vars = ["swhite", "sblack", "shisp", "sother"]

# List of all variables for which we want to compute means
all_vars = dat.columns

# Initialize an empty DataFrame to store results
summary_table = pd.DataFrame()

# Calculate the overall mean for each variable
overall_means = dat[all_vars].mean()
summary_table["Variable"] = all_vars
summary_table["Full Sample"] = overall_means.values

# Calculate the mean for each variable within each group
for group in group_vars:
    group_means = dat.loc[dat[group] == 1, all_vars].mean()
    summary_table[group.capitalize()] = group_means.values

# Add a row for "Number of Observations"
num_obs_row = pd.DataFrame({
    "Variable": ["Number of Observations"],
    "Full Sample": [dat.shape[0]],
    **{group.capitalize(): [dat.loc[dat[group] == 1].shape[0]] for group in group_vars}
})

# Append the "Number of Observations" row to the summary table
summary_table = pd.concat([summary_table, num_obs_row], ignore_index=True)

# Format the table for display
summary_table = summary_table.set_index("Variable")
print(summary_table)


                        Full Sample       Swhite       Sblack        Shisp  \
Variable                                                                     
sblack                     0.110555     0.000000     1.000000     0.000000   
shisp                      0.101606     0.000000     0.000000     1.000000   
swhite                     0.739142     1.000000     0.000000     0.000000   
sother                     0.048697     0.000000     0.000000     0.000000   
smale                      0.529613     0.521368     0.519048     0.585492   
sage                      41.010003    42.147792    39.183333    36.225389   
sempl                      0.695446     0.699786     0.688095     0.676166   
sincome                    2.164780     2.218305     1.935714     1.997409   
spop                       1.362727     1.271011     1.657143     1.652850   
daytime                    0.666491     0.680912     0.621429     0.642487   
inctype_lin                1.958410     1.957621     1.966667   

### summary table for assignment

In [33]:
# attach lanbels to data
labels = {
    'sblack': 'Black',
    'shisp': 'Hispanic',
    'swhite': 'White',
    'sother': 'Other race',
    'sage': 'Age',
    'sempl': 'Employed last week or not',
    'smale': 'Male',
    'spop': 'Population size of civilian\'s address',
    'sincome': 'Income (categorical)',
    'sbehavior': 'Behavior of civilian in encounter',
    'omajwhite': 'Officer unit majorly white',
    'omajother': 'Officer unit majorly other',
    'omajblack': 'Officer unit majorly black',
    'omajhisp': 'Officer unit majorly hispanic',
    'osplit': 'Officer unit split race',
    'daytime': 'Time of encounter',
    'year': 'Year',
    'inctype_lin': 'Incident type',
    'anyuseofforce_coded': 'Any use of force by officer'}

data_labeled = dat.rename(columns=labels)
data_labeled.columns
# table with summary statistics conditional on race and interaction
# a. make columns for each race and full sample
full_sample = dat
white_only = dat[dat['swhite'] == 1]
black_only = dat[dat['sblack'] == 1]
hispanic_only = dat[dat['shisp'] == 1]
other_only = dat[dat['sother'] == 1 ]
groups = [full_sample, white_only, black_only, hispanic_only, other_only]
group_names = ['Full sample', 'White only', 'Black only', 'Hispanic only', 'Other only']
variables = ['sage','sempl', 'smale','sincome', 'spop', 'sbehavior', 'omajwhite', 
             'omajblack', 'omajhisp', 'omajother', 'osplit', 'daytime', 
             'inctype_lin', 'anyuseofforce_coded']
variables_labeled = [labels[var] for var in variables]
#####################################################
# create empty dataframe to store summary statistics
summary_stats = pd.DataFrame(columns=group_names, index=variables_labeled)
# append two columns for p-values from t-tests between white vs black and hispanic vs white
summary_stats['p-value (Black vs White)'] = np.nan
summary_stats['p-value (Hispanic vs White)'] = np.nan
summary_stats.index.name = 'Variable'
for i, group in enumerate(groups):
    summary_stats.iloc[:, i] = group[variables].mean().round(2) #mean
    # t-test means of black vs white and hispanic vs white
    if group_names[i] == 'Black only':
        from scipy.stats import ttest_ind
        t_stat, p_val = ttest_ind(black_only[variables], white_only[variables], equal_var=False, nan_policy='omit')
        summary_stats['p-value (Black vs White)'] = p_val.round(2)
    if group_names[i] == 'Hispanic only':
        from scipy.stats import ttest_ind
        t_stat, p_val = ttest_ind(hispanic_only[variables], white_only[variables], equal_var=False, nan_policy='omit')
        summary_stats['p-value (Hispanic vs White)'] = p_val.round(2)

#####################################################
summary_stats.to_latex('summary_stats.tex', index=True, decimal=',', float_format="%.2f")

### prepare estimations

In [34]:
# Declare labels    
y_lab = 'anyuseofforce_coded'
#x_lab = ['const', 'sblack', 'shisp', 'sother']
#x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sempl', 'sincome', 'spop', 'sage', 'sagesq']
#x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sempl', 'sincome', 'spop', 'sage', 'sagesq', 'daytime', 'inctype_lin', 'omajblack', 'omajhisp', 'omajother']
x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sempl', 'sincome', 'spop', 'sage', 'sagesq', 'daytime', 'inctype_lin', 'omajblack', 'omajhisp', 'omajother', 'sbehavior']

dat['sage'] = dat['sage'] / 10
dat['sagesq'] = dat.sage * dat.sage 

# create extra variables 
N = dat.shape[0]
dat['const'] = np.ones((N,))

# Rebuild the dataset
dat = dat[[y_lab] + x_lab].copy()

# Check for missing data
assert dat.notnull().all(axis=1).all(), 'Missing values detected. Clean your data!'
dat.tail(5)

,anyuseofforce_coded,const,sblack,shisp,sother,smale,sempl,sincome,spop,sage,sagesq,daytime,inctype_lin,omajblack,omajhisp,omajother,sbehavior
3794,0,1.0,0,0,0,0,1,3,1,7.2,51.84,0,1,0,0,0,1
3795,0,1.0,0,0,0,0,0,2,1,7.1,50.41,1,2,0,0,0,0
3796,0,1.0,0,0,0,0,0,1,1,7.6,57.76,1,2,0,0,0,0
3797,0,1.0,0,0,0,0,0,3,4,7.9,62.41,1,2,0,0,0,0
3798,0,1.0,0,0,0,0,0,2,1,7.5,56.25,1,2,0,0,0,0


In [35]:
# descriptive of who's been victim to police violence
violence = dat[dat['anyuseofforce_coded']==1] 

# List of all variables for which we want to compute means
all_vars = violence.columns

# Initialize an empty DataFrame to store results
summary_table = pd.DataFrame()

# Calculate the overall mean for each variable
overall_means = violence[all_vars].mean()
summary_table["Variable"] = all_vars
summary_table["Full Sample"] = overall_means.values

# Format the table for display
summary_table = summary_table.set_index("Variable")
print(summary_table)

# Optional: Save the table to a CSV for further analysis
summary_table.to_latex('violence.csv')
# sep issues with omajblack and omajother


                     Full Sample
Variable                        
anyuseofforce_coded     1.000000
const                   1.000000
sblack                  0.157895
shisp                   0.315789
sother                  0.052632
smale                   0.789474
sempl                   0.473684
sincome                 2.052632
spop                    1.947368
sage                    3.078947
sagesq                 10.666842
daytime                 0.473684
inctype_lin             1.684211
omajblack               0.000000
omajhisp                0.052632
omajother               0.000000
sbehavior               0.526316


In [36]:
# Extract y and X
y = dat[y_lab].values
x = dat[x_lab].values
K = x.shape[1]

print(K)
print(np.shape(x))

16
(3799, 16)


## Estimate using probit

In [7]:
# Initialize starting values
theta0 = probit.starting_values(y, x)

# Estimate model with probit
probit_results = est.estimate(probit.q, theta0, y, x, cov_type='Sandwich')

Optimization terminated successfully.
         Current function value: 0.022253
         Iterations: 159
         Function evaluations: 2958
         Gradient evaluations: 174


In [ ]:
probit_tab = est.print_table(x_lab, probit_results, title=f'Probit, y = {y_lab}')
probit_tab

Optimizer succeeded after 159 iter. (2958 func. evals.). Final criterion:  0.02225.
Probit, y = anyuseofforce_coded


,theta,se,t
const,-2.6161,0.6954,-3.7622
sblack,0.2042,0.2988,0.6833
shisp,0.4162,0.2382,1.7468
sother,0.1051,0.4384,0.2397
smale,0.5619,0.2102,2.6736
sempl,-0.5001,0.2205,-2.2676
sincome,0.1022,0.1243,0.8216
spop,0.1988,0.0747,2.6626
sage,0.4861,0.3719,1.3072
sagesq,-0.0791,0.0499,-1.5856


### Test for misspecification

In [9]:
# White's information matrix test for model misspecification
whites_test = probit.whites_imt_probit(probit_results['theta'], y, x)
probit.print_test_stats(whites_test[0], whites_test[1])

White's Information Matrix Test for Probit Model Misspecification
------------------------------------------------------------------
Test Statistic: 2270.8817
P-value: 0.0000
Result: Reject the null hypothesis of correct model specification at the 5% significance level.


## Estimate using Logit

In [10]:
# Initialize starting values
theta0 = logit.starting_values(y, x)

# Estimate model with logit
logit_results = est.estimate(logit.q, theta0, y, x, cov_type='Sandwich')

Optimization terminated successfully.
         Current function value: 0.022389
         Iterations: 203
         Function evaluations: 3604
         Gradient evaluations: 212


In [11]:
logit_tab = est.print_table(x_lab, logit_results, title=f'Logit, y = {y_lab}')
logit_tab

Optimizer succeeded after 203 iter. (3604 func. evals.). Final criterion:  0.02239.
Logit, y = anyuseofforce_coded


,theta,se,t
const,-5.6105,1.9349,-2.8996
sblack,0.4362,0.8092,0.5390
shisp,0.9404,0.6142,1.5312
sother,-0.0945,1.3081,-0.0723
smale,1.1543,0.5733,2.0135
sempl,-1.1609,0.6149,-1.8880
sincome,0.2021,0.3418,0.5914
spop,0.4812,0.1903,2.5287
sage,1.3028,1.1759,1.1079
sagesq,-0.2207,0.1652,-1.3362


### Test for misspecification

In [12]:
# White's information matrix test for model misspecification
whites_test = logit.whites_imt_logit(logit_results['theta'], y, x)
logit.print_test_stats(whites_test[0], whites_test[1])

White's Information Matrix Test for Probit Model Misspecification
------------------------------------------------------------------
Test Statistic: 2412.3541
P-value: 0.0000
Result: Reject the null hypothesis of correct model specification at the 5% significance level.


## Average partial effects

### Probit

In [13]:
# Estimating the average partial effects using the probit
indices = [x_lab.index('sblack'), x_lab.index('shisp'), x_lab.index('sother')]  
labels = ['sblack', 'shispanic', 'sother'] 
res_probit = probit.properties(x, probit_results['theta'],print_out = True,se=True,indices=indices, labels = labels)
res_probit

,Estimate
sblack,0.002
shispanic,0.006
sother,0.001


In [15]:
#bootstrap SEs
se = probit.bootstrap(y, x, probit_results['theta'], indices, nB=100)
formatted_se =pd.DataFrame(se, index=["Black", "Hispanic", "Other"], columns=["Boostrapped SE"]).round(6)
formatted_se

Optimization terminated successfully.
         Current function value: 0.018157
         Iterations: 106
         Function evaluations: 1853
         Gradient evaluations: 109
Optimization terminated successfully.
         Current function value: 0.025581
         Iterations: 120
         Function evaluations: 2074
         Gradient evaluations: 122
Optimization terminated successfully.
         Current function value: 0.025581
         Iterations: 120
         Function evaluations: 2074
         Gradient evaluations: 122
Optimization terminated successfully.
         Current function value: 0.015862
         Iterations: 134
         Function evaluations: 2312
         Gradient evaluations: 136
Optimization terminated successfully.
         Current function value: 0.015862
         Iterations: 134
         Function evaluations: 2312
         Gradient evaluations: 136
Bootstrap iteration 2 failed: Singular matrix
Bootstrap iteration 2 failed: Singular matrix
Optimization terminated succ

,Boostrapped SE
Black,0.004004
Hispanic,0.004275
Other,0.004757


### Logit

In [16]:
# Estimating the average partial effects using the logit
indices = [x_lab.index('sblack'), x_lab.index('shisp'), x_lab.index('sother')]  
labels = ['sblack', 'shispanic', 'sother']  
res_logit = logit.properties(x, logit_results['theta'],print_out = True,se=True,indices=indices, labels = labels)
res_logit

,Estimate
sblack,0.002
shispanic,0.005
sother,-0.000


In [18]:
#bootstrap SEs
se = logit.bootstrap(y, x, logit_results['theta'], indices, nB=100)
formatted_se =pd.DataFrame(se, index=["Black", "Hispanic", "Other"], columns=["Boostrapped SE"]).round(6)
formatted_se

Optimization terminated successfully.
         Current function value: 0.021399
         Iterations: 149
         Function evaluations: 2550
         Gradient evaluations: 150
Bootstrap iteration 0 failed: Singular matrix
Optimization terminated successfully.
         Current function value: 0.025381
         Iterations: 90
         Function evaluations: 1547
         Gradient evaluations: 91
Optimization terminated successfully.
         Current function value: 0.025381
         Iterations: 90
         Function evaluations: 1547
         Gradient evaluations: 91
Optimization terminated successfully.
         Current function value: 0.019848
         Iterations: 185
         Function evaluations: 3162
         Gradient evaluations: 186
Optimization terminated successfully.
         Current function value: 0.019848
         Iterations: 185
         Function evaluations: 3162
         Gradient evaluations: 186
Optimization terminated successfully.
         Current function value: 0.01613

,Boostrapped SE
Black,0.004208
Hispanic,0.005370
Other,0.004330


## Partial Effects

#### Defining different fixed vectors

In [196]:
#means of the regressors
print(f"{np.mean(dat['sage']):.2f}")
print(f"{np.mean(dat['sagesq']):.2f}")
print(f"{np.mean(dat['sincome']):.2f}")
print(f"{np.mean(dat['spop']):.2f}")

4.10
19.42
2.16
1.36


In [204]:
# Original vector
# make vector of stereotypical white young man 
x_lab = ['const', 'sblack', 'shisp', 'sother', 
         'smale', 'sage', 'sempl', 'sincome',
         'spop', 'daytime', 'inctype_lin', 'omajblack',
         'omajhisp', 'omajother', 'sbehavior','sagesq']

x_me = np.array([1, 0, 0, 0,
                 1, 2.5, 0, 1,
                 4, 5, 1, 0, 
                 0, 0, 1, 6.25]).reshape(1, -1)

pd.DataFrame(x_me, columns=x_lab, index=['x_me'])

,const,sblack,shisp,sother,smale,sage,sempl,sincome,spop,daytime,inctype_lin,omajblack,omajhisp,omajother,sbehavior,sagesq
x_me,1.0,0.0,0.0,0.0,1.0,2.5,0.0,1.0,4.0,5.0,1.0,0.0,0.0,0.0,1.0,6.25


In [198]:
# make vector of stereotypical white old woman
x_lab = ['const', 'sblack', 'shisp', 'sother', 
         'smale', 'sage', 'sempl', 'sincome',
         'spop', 'daytime', 'inctype_lin', 'omajblack',
         'omajhisp', 'omajother', 'sbehavior','sagesq']

x_me = np.array([1, 0, 0, 0,
                 0, 4.5, 1, 3,
                 2, 2, 1, 0, 
                 0, 0, 1, 20.25]).reshape(1, -1)

pd.DataFrame(x_me, columns=x_lab, index=['x_me'])


,const,sblack,shisp,sother,smale,sage,sempl,sincome,spop,daytime,inctype_lin,omajblack,omajhisp,omajother,sbehavior,sagesq
x_me,1.0,0.0,0.0,0.0,0.0,4.5,1.0,3.0,2.0,2.0,1.0,0.0,0.0,0.0,1.0,20.25


### Switcing race 

In [205]:
#k=1: black 
#k=2: hispanic
#k=3: other

k = 1
x_me2 = x_me.copy()
x_me2[:, k] = 1   
pd.DataFrame(x_me2, columns=x_lab, index=['x_me2'])

,const,sblack,shisp,sother,smale,sage,sempl,sincome,spop,daytime,inctype_lin,omajblack,omajhisp,omajother,sbehavior,sagesq
x_me2,1.0,1.0,0.0,0.0,1.0,2.5,0.0,1.0,4.0,5.0,1.0,0.0,0.0,0.0,1.0,6.25


In [206]:
# fetch coef from probit model
b_pr = probit_tab.theta.values
# find marginal effect as difference in predicted probabilities
me_race_pr = probit.G(x_me2@b_pr) - probit.G(x_me@b_pr) 

In [207]:
# calculate gradient
gx0 = norm.pdf(x_me@b_pr)
gx2 = norm.pdf(x_me2@b_pr)
grad_d_pr = gx2*x_me2 - gx0*x_me

In [208]:
# define function to calculate std. errors
def get_se(grad, cov):
    # calculate variance-covariance matrix
    cov_me = grad@cov@grad.T
    # return squareroot of variance = se
    return np.sqrt(np.diag(cov_me))

se_d_pr = get_se(grad_d_pr, probit_results['cov'])

In [209]:
# create table with results
me_dict = {'Marginal Effect': me_race_pr[0],
           's.e.':            se_d_pr}
tab = pd.DataFrame(me_dict)
tab['t'] = tab['Marginal Effect'] / tab['s.e.']
tab.index.name = 'Var'
tab.round(6)

,Marginal Effect,s.e.,t
Var,,,
0,0.002253,0.008689,0.259265


### Logit

In [ ]:
# fetch coef from logit model
b_lg = logit_tab.theta.values
# cal diff in predicted probabilities
me_race_lg = logit.G(x_me2@b_lg) - logit.G(x_me@b_lg)

In [ ]:
# Compute the logistic function exponential terms for x_me2 and x_me
exp_x0_b = np.exp(-(x_me@b_lg))
exp_x2_b = np.exp(-(x_me2@b_lg))
grad_d_lg = (x_me2 * exp_x2_b)/ (1 + exp_x2_b)**2 - (x_me * exp_x0_b)/ (1 + exp_x0_b)**2
se_d_lg = get_se(grad_d_lg, logit_results['cov'])

In [ ]:
# find p value for marginal effects
from scipy.stats import norm
p_values = 2 * (1 - norm.cdf(np.abs(tab['t'])))
tab['p-value'] = p_values

In [ ]:
tab.to_latex('me_probit_youngman_hisp.tex')

In [ ]:
me_dict = {'Marginal Effect': me_race_lg[0],
           's.e.':            se_d_lg}
tab = pd.DataFrame(me_dict)
tab['t'] = tab['Marginal Effect'] / tab['s.e.']
tab.index.name = 'Var'
tab.round(6)

In [ ]:
p_values = 2 * (1 - norm.cdf(np.abs(tab['t'])))
tab['p-value'] = p_values

In [ ]:
tab.to_latex('me_logit_youngman_hisp.tex')

### table with probit and logit results for marginal effects

In [ ]:
files = [
    'me_probit_youngman.tex',
    'me_probit_youngman_hisp.tex',
    'me_logit_youngman.tex',
    'me_logit_youngman_hisp.tex',
    'me_probit_old_woman.tex',
    'me_probit_old_woman_hisp.tex',
    'me_logit_old_woman.tex',
    'me_logit_old_woman.tex'
]

with open('combined_table.tex', 'w') as outfile:
    outfile.write(r'\begin{table}[ht!]' + '\n')
    outfile.write(r'\centering' + '\n')
    outfile.write(r'\caption{Marginal Effects from Probit and Logit Models}' + '\n')
    outfile.write(r'\begin{tabular}{lccc}' + '\n')
    outfile.write(r'\toprule' + '\n')
    
    panels = ['Panel A: Probit, Young Man',
              'Panel B: Probit, Young Man Hispanic',
              'Panel C: Logit, young man',
              'Panel D: Logit, young man, hispanic',
              'Panel E: Probit, Old Woman',
              'Panel F: Probit, Old Woman, hispanic',
              'Panel G: Logit, Old Woman',
              'Panel H: Logit, Old Woman, hispanic']

    for panel_name, fname in zip(panels, files):
        outfile.write(r'\midrule' + '\n')
        outfile.write(r'\multicolumn{4}{c}{\textbf{' + panel_name + r'}} \\' + '\n')
        outfile.write(r'\midrule' + '\n')
        with open(fname) as infile:
            content = infile.read()
            outfile.write(content + '\n')
    
    outfile.write(r'\bottomrule' + '\n')
    outfile.write(r'\end{tabular}' + '\n')
    outfile.write(r'\end{table}' + '\n')

### LM test of probit model

In [37]:
# conduct LM tests of different model formulations
#x_lab = ['const', 'sblack', 'shisp', 'sother']
#x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sempl', 'sincome', 'spop', 'sage', 'sagesq']
#x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sempl', 'sincome', 'spop', 'sage', 'sagesq', 'daytime', 'inctype_lin', 'omajblack', 'omajhisp', 'omajother']
#x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sempl', 'sincome', 'spop', 'sage', 'sagesq', 'daytime', 'inctype_lin', 'omajblack', 'omajhisp', 'omajother', 'sbehavior']

y_lab = 'anyuseofforce_coded'
x_lab_restricted = ['const', 'sblack', 'shisp', 'sother']
x_restricted = dat[x_lab_restricted].values

# a. Estimate restricted model
theta0 = probit.starting_values(y, x_restricted)
results_restricted = est.estimate(probit.q, theta0, y, x_restricted)
theta_restricted = results_restricted['theta']

# b. Prepare the additional variables to test
# first augmentation
x_lab_additional_1 = ['smale', 'sempl', 'sincome', 'spop', 'sage', 'sagesq']
x_additional_1 = dat[x_lab_additional_1].values
# second augmentation
x_lab_additional_2 = ['daytime', 'inctype_lin', 'omajblack', 'omajhisp', 'omajother']
x_additional_2 = dat[x_lab_additional_2].values
# third augmentation
x_lab_additional_3 = ['sbehavior']
x_additional_3 = dat[x_lab_additional_3].values

# c. Run the LM test
# first augmentation
stat, pval, df = probit.LM_test(y, x_restricted, theta_restricted, x_additional_1)
# second
stat2, pval2, df2 = probit.LM_test(y, x_restricted, theta_restricted, x_additional_2)
#third
stat3, pval3, df3 = probit.LM_test(y, x_restricted, theta_restricted, x_additional_3)
# d. Print results
probit.print_LM_test(stat, pval, df, var_names=x_lab_additional_1)
probit.print_LM_test(stat2, pval2, df2, var_names=x_lab_additional_2)
probit.print_LM_test(stat3, pval3, df3, var_names=x_lab_additional_3)

Optimization terminated successfully.
         Current function value: 0.030440
         Iterations: 37
         Function evaluations: 195
         Gradient evaluations: 39

Lagrange Multiplier (LM) Test for Variable Inclusion
Testing inclusion of: smale, sempl, sincome, spop, sage, sagesq
Degrees of freedom: 6
LM Test Statistic: 23.5037
P-value: 0.0006
------------------------------------------------------------
Result: Reject H0 at 1% level - additional variables are significant

Lagrange Multiplier (LM) Test for Variable Inclusion
Testing inclusion of: daytime, inctype_lin, omajblack, omajhisp, omajother
Degrees of freedom: 5
LM Test Statistic: 2.4918
P-value: 0.7777
------------------------------------------------------------
Result: Fail to reject H0 - additional variables are not significant

Lagrange Multiplier (LM) Test for Variable Inclusion
Testing inclusion of: sbehavior
Degrees of freedom: 1
LM Test Statistic: 58.9636
P-value: 0.0000
----------------------------------------

### LM test of logit model

In [38]:
# conduct LM tests of different model formulations
#x_lab = ['const', 'sblack', 'shisp', 'sother']
#x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sempl', 'sincome', 'spop', 'sage', 'sagesq']
#x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sempl', 'sincome', 'spop', 'sage', 'sagesq', 'daytime', 'inctype_lin', 'omajblack', 'omajhisp', 'omajother']
#x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sempl', 'sincome', 'spop', 'sage', 'sagesq', 'daytime', 'inctype_lin', 'omajblack', 'omajhisp', 'omajother', 'sbehavior']

y_lab = 'anyuseofforce_coded'
x_lab_restricted = ['const', 'sblack', 'shisp', 'sother']
x_restricted = dat[x_lab_restricted].values

# a. Estimate restricted model
theta0 = logit.starting_values(y, x_restricted)
results_restricted = est.estimate(logit.q, theta0, y, x_restricted)
theta_restricted = results_restricted['theta']

# b. Prepare the additional variables to test
# first augmentation
x_lab_additional_1 = ['smale', 'sempl', 'sincome', 'spop', 'sage', 'sagesq']
x_additional_1 = dat[x_lab_additional_1].values
# second augmentation
x_lab_additional_2 = ['daytime', 'inctype_lin', 'omajblack', 'omajhisp', 'omajother']
x_additional_2 = dat[x_lab_additional_2].values
# third augmentation
x_lab_additional_3 = ['sbehavior']
x_additional_3 = dat[x_lab_additional_3].values

# c.. Run the LM test
# first augmentation
stat, pval, df = logit.LM_test(y, x_restricted, theta_restricted, x_additional_1)
# second
stat2, pval2, df2 = logit.LM_test(y, x_restricted, theta_restricted, x_additional_2)
#third
stat3, pval3, df3 = logit.LM_test(y, x_restricted, theta_restricted, x_additional_3)
# d. Print results
logit.print_LM_test(stat, pval, df, var_names=x_lab_additional_1)
logit.print_LM_test(stat2, pval2, df2, var_names=x_lab_additional_2)
logit.print_LM_test(stat3, pval3, df3, var_names=x_lab_additional_3)

Optimization terminated successfully.
         Current function value: 0.030440
         Iterations: 51
         Function evaluations: 265
         Gradient evaluations: 53

Lagrange Multiplier (LM) Test for Variable Inclusion
Testing inclusion of: smale, sempl, sincome, spop, sage, sagesq
Degrees of freedom: 6
LM Test Statistic: 23.5757
P-value: 0.0006
------------------------------------------------------------
Result: Reject H0 at 1% level - additional variables are significant

Lagrange Multiplier (LM) Test for Variable Inclusion
Testing inclusion of: daytime, inctype_lin, omajblack, omajhisp, omajother
Degrees of freedom: 5
LM Test Statistic: 2.5265
P-value: 0.7725
------------------------------------------------------------
Result: Fail to reject H0 - additional variables are not significant

Lagrange Multiplier (LM) Test for Variable Inclusion
Testing inclusion of: sbehavior
Degrees of freedom: 1
LM Test Statistic: 56.7351
P-value: 0.0000
----------------------------------------